In [1]:
import pandas as pd
import numpy as np
import torch
from mm.datasets import generate_data

In [2]:
df = generate_data(5)
df.treatment = df.treatment
df

,treatment,hospital,outcome
0,0,3,-1.870661
1,1,1,3.062477
2,1,1,4.757326
3,0,0,0.038551
4,0,1,1.812503


In [16]:
def tensor(x, shape=None, dtype=float):
    t = torch.tensor(x).to(float)
    if type(shape) != int and shape is not None:
        shape = tuple(shape)
    if shape is not None:
        if t.shape != shape:
            try:
                t = t.reshape(shape)
            except:
                raise ValueError(f"Cannot reshape {t.shape} to {shape}")
    return t

# Problem Setup


The Model

$Y \sim N(X \beta + o, \sigma^2 W^{-1})$

$(Y | B = b) \sim N(X \beta + Z b + o, \sigma^2 W^{-1})$

$B \sim N(0, \Sigma)$

| name | shape | desc |
| --- | --- | --- |
| $X$ | $n \times \rho$ | fixed effect design matrix |
| $\beta$ | $\rho \times 1$ | fixed effect parameters |
| $Z$ | $n \times p$ | random effect design matrix |
| $b$ | $p \times 1$ | random effect parameters |
| $o$ | $n \times 1$ | prior offset terms |
| $\sigma^2$ | scalar | covariance scale parameter |
| $W^{-1}$ | $n \times n$ | covariance structure matrix |

The main problem with this setup as stated is that $\Sigma$, as a covariance matrix, must be nonsingular. We are going to be doing some kind of gradient descent over the elements in sigma though, and it would be convenient to not have to explicitly enforce this constraint during the optimization process. So, instead of modeling $B$ as a a normal with a given covariance, we model it as a linear transform of a unit-variance random variable $U$.

$U \sim N(0, \sigma^2 I)$

$B = \Lambda_\theta U$

Thus,

$\Sigma_\theta = \sigma^2 \Lambda_\theta \Lambda_\theta^T$

We can then restate the model as,

$\mu_{y | U = u} = X \beta + Z \Lambda_\theta u + o$

$Y | U = N(\mu_{y | U = u}, \sigma^2 W^{-1})$

Deriving the matrix $Z$ involves a lot of index chasing, even though the idea is not that complicated. Basically, if we want to model the random effect of some variable $x$ with a grouping variable $g$, we have to split the $x$ column into a bunch of separate column, each corresponding to a unique value of $g$. For each of the columns, a cell will contain the corresponding unit's value of $x$ if and only if that column corresponds to the unit's value of $g$. Otherwise, it will be zero.

The relative covariance factor $\Lambda_\theta$ will be a block diagonal matrix that specifies zero covariance between parameters for different groups. Inside each group, it will be a lower-diagonal matrix where each non-zero cell is a learnable parameter.

- Are these learnable parameters different for each group? Idk

It's important to allow correlation between random effect parameters inside each group because this makes the model invariant to additive shifts in the independent variable.

- Why? Idk

Note that both $Z$ and $\Lambda$ are gonna be pretty sparse.

# Side Quest

Let's consider what our optimization problem is going to look like. We have distributions over both $y_{\text{obs}}$ and $U$. Since they're both normal, the likelihoods are pretty stratiforward.

Since $U$ has unit variance, $\mathcal{L}(U = u) \propto -||u||^2$.

Similarly, $\mathcal{L}(y = y_{\text{obs}}) \propto -||W^{1/2}[y_{\text{obs}} - \mu_{U = u}]||^2$.

We are faced with the minimization problem

$\hat{u}, \hat{\beta} = \underset{u, \beta}{\mathrm{argmin}} ( r^2 )$

For

$r^2 = ||W^{1/2}[y_{\text{obs}} - \mu_{U = u}]||^2 + ||u||^2$

Note that what we are trying to do here is just minimizing the squared length of the vector $\rho^2$. This means that we can use the "pseudo-data" approach and think of this as an OLS problem. First, we rephrase it using matrix notation,

$$
r^2 =
\bigg|\bigg|
\begin{bmatrix}
W^{1/2}(y_{\text{obs}} - o) \\
0
\end{bmatrix} -
\begin{bmatrix}
W^{1/2}Z\Lambda_\theta & W^{1/2}X \\
I & 0
\end{bmatrix}
\begin{bmatrix}
u \\ b
\end{bmatrix}
\bigg|\bigg|^2
$$

The top row of blocks is finding the penalty for $y_{\text{obs}}$, and the bottom row is finding the penalty for $u$. One the left side of the difference, we have the means of the two distributions, and on the right side, we have the values conjectured by our parameters.

(somehow? by GLS?) We determine that the parameters of best fit are given by

$$
\begin{bmatrix}
\Lambda_\theta^T Z^T W (y_{\text{obs}} - o)\\
X^T W (y_{\text{obs}} - o)
\end{bmatrix} =
\begin{bmatrix}
\Lambda_\theta^T Z^T W Z \Lambda_\theta + I & \Lambda_\theta^T Z^T W X \\
(\Lambda_\theta^T Z^T W X)^T & X^T W X
\end{bmatrix}
\begin{bmatrix}
\mu_{U | y = y_{\text{obs}}} \\
\hat{\beta_\theta}
\end{bmatrix}
$$

We can cholesky decompose this bad boy.

$$
\begin{bmatrix}
\Lambda_\theta^T Z^T W Z \Lambda_\theta + I & \Lambda_\theta^T Z^T W X \\
(\Lambda_\theta^T Z^T W X)^T & X^T W X
\end{bmatrix} = 
\begin{bmatrix}
L_\theta & 0 \\
R^T_{ZX} & R^T_X
\end{bmatrix}
\begin{bmatrix}
L^T_\theta & R_{ZX} \\
0 & R_X
\end{bmatrix}
$$

Then, apparently,

$$
r^2 = \text{min}(r^2) +
||
L^T_\theta (u - \mu_{U | y = y_{\text{obs}}}) +
R_{ZX} (\beta - \hat{\beta_\theta})
||^2 +
|| R_X(\beta - \hat{\beta_\theta}) ||^2
$$

Which is apparently useful.

# Solving for Likelihood

From the assumptions of the model, we have,

$$
f_{y | U}(y_{\text{obs}} | u) =
{|W|^{1/2} \over (2 \pi \sigma^2)^{n/2}}
\exp \bigg( {
    -||W^{1/2}(y_{\text{obs}} - \mu_{y | U = u})||^2 \over
    2 \sigma^2
} \bigg)
$$

$$
f_U(u) = {1 \over (2 \pi \sigma^2)^{q/2}}
\exp \bigg({
    -||u||^2 \over 2\sigma^2
}\bigg)
$$

This gives us,

$$
f_{y, U}(y_{\text{obs}}, u) = f_{y | U}(y_{\text{obs}} | u) f_U(u) =
{|W|^{1/2} \over (2\pi \sigma^2)^{(n + q)/2}}
\exp \bigg(
    {-r^2 \over 2\sigma^2}
\bigg)
$$

From which we can derive,

$$
f_y{y_{\text{obs}}} = \int f_{y, U}(y_{\text{obs}}, u) du
$$

$$
f_{U | y}(u | y_{\text{obs}}) = {f_{y, U}(y_{\text{obs}}, u) \over f_y{y_{\text{obs}}}}
$$

The marginal distribution of $y$ is what we care about. We can expand and factor out the $u$ terms.

$f_y(y_{\text{obs}})$

$= \int f_{y, U}(y_{\text{obs}}, u) du$

$
= \int
    {|W|^{1/2} \over (2\pi \sigma^2)^{(n + q)/2}}
    \exp \bigg(
        {-r^2 \over 2\sigma^2}
    \bigg)
du
$

$
= \int
    {|W|^{1/2} \over (2\pi \sigma^2)^{(n + q)/2}}
    \exp \bigg(
        {
            -(\text{min}(r^2) +
            ||
            L^T_\theta (u - \mu_{U | y = y_{\text{obs}}}) +
            R_{ZX} (\beta - \hat{\beta_\theta})
            ||^2 +
            || R_X(\beta - \hat{\beta_\theta}) ||^2)
            \over
            2\sigma^2
        }
    \bigg)
du
$

$
= \int
    {|W|^{1/2} \over (2\pi \sigma^2)^{(n + q)/2}}
    \exp \bigg(
        {
            - \text{min}(r^2)
            - || R_X(\beta - \hat{\beta_\theta}) ||^2
            \over
            2\sigma^2
        }
    \bigg)
    \exp \bigg(
        {
            -
            ||
            L^T_\theta (u - \mu_{U | y = y_{\text{obs}}}) +
            R_{ZX} (\beta - \hat{\beta_\theta})
            ||^2
            \over
            2\sigma^2
        }
    \bigg)
du
$

$= {|W|^{1/2} \over (2\pi \sigma^2)^{(n + q)/2}}
\exp \bigg(
    {
        - \text{min}(r^2)
        - || R_X(\beta - \hat{\beta_\theta}) ||^2
        \over
        2\sigma^2
    }
\bigg)
\int
    \exp \bigg(
        {
            -
            ||
            L^T_\theta (u - \mu_{U | y = y_{\text{obs}}}) +
            R_{ZX} (\beta - \hat{\beta_\theta})
            ||^2
            \over
            2\sigma^2
        }
    \bigg)
du
$

We now let,

$v = L^T_\theta (u - \mu_{U | y = y_{\text{obs}}}) + R_{ZX} (\beta - \hat{\beta_\theta})$

And then, via change of variables, the integral becomes,

$= {|W|^{1/2} \over (2\pi \sigma^2)^{(n + q)/2}}
\exp \bigg(
    {
        - \text{min}(r^2)
        - || R_X(\beta - \hat{\beta_\theta}) ||^2
        \over
        2\sigma^2
    }
\bigg)
\int
    \exp \bigg(
        {- || v ||^2 \over 2\sigma^2}
    \bigg)
    |L_\theta|^{-1}
dv
$

Which is apparently equal to,

$= {|W|^{1/2} |L_\theta|^{-1} \over (2\pi \sigma^2)^{n/2}}
\exp \bigg(
    {
        - \text{min}(r^2)
        - || R_X(\beta - \hat{\beta_\theta}) ||^2
        \over
        2\sigma^2
    }
\bigg)
$

# Maximum Likelihood Solution

We define,

$$
\mathcal{L}(\theta, \beta, \sigma^2 | y_{\text{obs}}) = \log f_y (y_{\text{obs}})
$$

Then,

$
-2\mathcal{L}(\theta, \beta, \sigma^2 | y_{\text{obs}}) = -2\log f_y (y_{\text{obs}})
$

$
= -2\log(
    {|W|^{1/2} |L_\theta|^{-1} \over (2\pi \sigma^2)^{n/2}}
    \exp \bigg(
        {
            - \text{min}(r^2)
            - || R_X(\beta - \hat{\beta_\theta}) ||^2
            \over
            2\sigma^2
        }
    \bigg)
)
$

$
= -2\log({|W|^{1/2} |L_\theta|^{-1} \over (2\pi \sigma^2)^{n/2}})
-2 {
    - \text{min}(r^2)
    - || R_X(\beta - \hat{\beta_\theta}) ||^2
    \over
    2\sigma^2}
$

$
= -2\log(|W|^{1/2} |L_\theta|^{-1}) + 2 \log((2\pi \sigma^2)^{n/2})
-2 {
    - \text{min}(r^2)
    - || R_X(\beta - \hat{\beta_\theta}) ||^2
    \over
    2\sigma^2}
$

$
= \log({|L_\theta|^{-1} \over |W|}) + n \log(2\pi \sigma^2) + {
    \text{min}(r^2)
    \over
    \sigma^2
} - {
|| R_X(\beta - \hat{\beta_\theta}) ||^2
    \over
    \sigma^2
}
$

To find the maximum likelihood solution, we just minimize this. We can first minimize with respect to $\beta$ by noticing that the only term containing $\beta$ is minimized when $\beta = \hat{\beta_\theta}$, yielding,

$
\log({|L_\theta|^{-1} \over |W|}) + n \log(2\pi \sigma^2) + {
    \text{min}(r^2)
    \over
    \sigma^2
}
$

We then take the derivative with respect to $\sigma^2$ and set the result to zero.

$
{n \over \hat{\sigma}^2} + {
    \text{min}(r^2)
    \over
    \hat{\sigma}^4
} = 0
$

$
\hat{\sigma}^2 = {\text{min}(r^2) \over n}
$

This result in the likelihood,

$
\log({|L_\theta|^{-1} \over |W|}) + n \log(2\pi {\text{min}(r^2) \over n}) + {
    \text{min}(r^2)n
    \over
    \text{min}(r^2)
}
$

$
= \log({|L_\theta|^{-1} \over |W|}) + n \big[ \log(2\pi {\text{min}(r^2) \over n}) + 1 \big]
$

# Restricted Maximum Likelihood Solution

$
\int f_y(y) d\beta = \int
{|W|^{1/2} |L_\theta|^{-1} \over (2\pi \sigma^2)^{n/2}}
\exp \bigg(
    {
        - \text{min}(r^2)
        - || R_X(\beta - \hat{\beta_\theta}) ||^2
        \over
        2\sigma^2
    }
\bigg)
d\beta
$

$
= \int
{|W|^{1/2} |L_\theta|^{-1} \over (2\pi \sigma^2)^{n/2}}
\exp \bigg(
    {
        - \text{min}(r^2)
        \over
        2\sigma^2
    }
\bigg)
\exp \bigg(
    {
        - || R_X(\beta - \hat{\beta_\theta}) ||^2
        \over
        2\sigma^2
    }
\bigg)
d\beta
$

$
= {|W|^{1/2} |L_\theta|^{-1} \over (2\pi \sigma^2)^{n/2}}
\exp \bigg(
    {
        - \text{min}(r^2)
        \over
        2\sigma^2
    }
\bigg)
\int
\exp \bigg(
    {
        - || R_X(\beta - \hat{\beta_\theta}) ||^2
        \over
        2\sigma^2
    }
\bigg)
d\beta
$

Let,

$v = R_X(\beta - \hat{\beta_\theta})$

For this, the jacobian determinant is $|R_X|$. Substituting into the above, we see that,

$
\int f_y(y) d\beta
= {|W|^{1/2} |L_\theta|^{-1} \over (2\pi \sigma^2)^{n/2}}
\exp \bigg(
    {
        - \text{min}(r^2)
        \over
        2\sigma^2
    }
\bigg)
\int
\exp \bigg(
    {
        - || v ||^2
        \over
        2\sigma^2
    }
\bigg)
|R_X|^{-1}
dv
$

$
= {|W|^{1/2} |L_\theta|^{-1} |R_X|^{-1} \over (2\pi \sigma^2)^{(n - p)/2}}
\exp \bigg(
    {
        - \text{min}(r^2)
        \over
        2\sigma^2
    }
\bigg)
$

Let,

$
-2 \mathcal{L}_R = -2 \log \big( \int f_y(y) d\beta \big)
$

Then,

$
= -2 \log \bigg[ {|W|^{1/2} |L_\theta|^{-1} |R_X|^{-1} \over (2\pi \sigma^2)^{(n - p)/2}}
\exp \bigg(
    {
        - \text{min}(r^2)
        \over
        2\sigma^2
    }
\bigg)
\bigg]
$

$
= -2 \log \bigg[ |W|^{1/2} |L_\theta|^{-1} |R_X|^{-1} \bigg] +
2 \log((2\pi \sigma^2)^{(n - p)/2}) - 2 {
    - \text{min}(r^2)
    \over
    2\sigma^2
}
$

$
= \log \bigg({|L_\theta|^2 |R_X|^2 \over |W|} \bigg) +
(n - p) \log(2\pi \sigma^2) + {
    \text{min}(r^2)
    \over
    \sigma^2
}
$

We find that

- steps?

$
\hat{\sigma^2_\theta} = {\min(r^2) \over n - p}
$

If a estimate for $\beta$ is required, it's possible to find it using maximum likelihood while fixing the value of $\theta$ to the REML estimate. This is kind of ad hoc, but it works.

This results in the final likelihood,

$
-2 \mathcal{L}_R
= \log \bigg({|L_\theta|^2 |R_X|^2 \over |W|} \bigg) +
(n - p) \log(2\pi {\min(r^2) \over n - p}) + {
    \text{min}(r^2)
    \over
    \big( {\min(r^2) \over n - p} \big)
}
$

$
-2 \mathcal{L}_R
= \log \bigg({|L_\theta|^2 |R_X|^2 \over |W|} \bigg) +
(n - p) \log(2\pi {\min(r^2) \over n - p}) + (n - p)
$

$
-2 \mathcal{L}_R
= \log \bigg({|L_\theta|^2 |R_X|^2 \over |W|} \bigg) +
(n - p) \big( \log(2\pi {\min(r^2) \over n - p}) + 1 \big)
$

In [26]:
outcome_var = "outcome"
fe_vars = ["treatment"]
re_vars = ["hospital"]

n = df.shape[0] # number of observations
k = 1 # number of random effect terms in formula
f = 1 # number of fixed effect terms

l = df[re_vars].drop_duplicates().shape[0] # number of groups
p = 1 # number of effects per group
q = p * l # number of re parameters
m = int((p + 1) * p / 2) # number of covariance parameters

y = tensor(df.outcome.values, (n, 1))
X = tensor(df[fe_vars].values, (n, f))
o = torch.zeros_like(y)
W = torch.eye(len(y)).to(float)

In [27]:
class RandomEffect:
    group_id_col_name = "RE_GROUP_ID"
    def __init__(self, df, effects, groups):
        self.effects = effects
        self.groups = groups

        # create group indicator matrix
        self.group_df = (df[self.groups]
            .drop_duplicates()
            .reset_index(drop=True)
            .reset_index(names=RandomEffect.group_id_col_name))
        
        self.all_levels = tensor(self.group_df[RandomEffect.group_id_col_name].values, l)
        self.df_levels = tensor(
            pd.merge(df[groups], self.group_df, on=self.groups, how="left")
            [RandomEffect.group_id_col_name]
            .values,
            (n, 1)
        )
        assert not torch.any(torch.isnan(self.df_levels))
        
        self.df_levels_indicator_matrix = tensor(pd.get_dummies(self.df_levels.reshape(-1)).values, (n, l))

        # create design matrix
        effect_dm = {
            effect:
                self.df_levels_indicator_matrix if effect == 1
                else self.df_levels_indicator_matrix * tensor(df[effect].values, (n, 1))
            for effect in self.effects
            }
        
        self.dm = tensor(torch.hstack([dm.T for dm in effect_dm.values()]).reshape(-1, n).T, (n, q))

        # create covariance parameters
        self.cv_params = torch.randn(m)
    
    def __lt__(self, other):
        return (self.l, self.groups, self.effects) < (other.l, other.groups, other.effects)
    
    def relative_covariance_factor(self):
        cv_template = torch.zeros(p, p)
        cv_template[*torch.tril_indices(p, p)] = self.cv_params
        cv = torch.block_diag(*(l*[cv_template]))
        return tensor(cv, (q, q))

re = RandomEffect(df, effects=[1], groups=["hospital"])

/var/folders/l_/zjyw1l0d37x3lbbq6x4vvs_00000gn/T/ipykernel_10037/3172120815.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(x).to(float)


In [35]:
rcf = re.relative_covariance_factor()

L = torch.linalg.cholesky((rcf.T @ re.dm.T @ W @ re.dm @ rcf) + torch.eye(rcf.shape[0]).to(float))
cu = torch.linalg.solve(L, torch.linalg.solve(L, rcf @ re.dm.T @ W @ y))
R_zx = torch.linalg.solve(L, torch.linalg.solve(L, rcf @ re.dm.T @ W @ X))
RXtRX = X.T @ W @ X - 

/var/folders/l_/zjyw1l0d37x3lbbq6x4vvs_00000gn/T/ipykernel_10037/3172120815.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(x).to(float)


tensor([[ 0.0000],
        [-0.5772],
        [ 0.0000]], dtype=torch.float64)

In [21]:
foo = torch.arange(12).reshape(3, 4).T
bar = torch.arange(12).reshape(3, 4).T + 12

torch.hstack([foo.T, bar.T]).reshape(-1, 4).T

tensor([[ 0, 12,  4, 16,  8, 20],
        [ 1, 13,  5, 17,  9, 21],
        [ 2, 14,  6, 18, 10, 22],
        [ 3, 15,  7, 19, 11, 23]])

In [4]:
W = np.eye(n)
o = np.ones(n).reshape(-1, 1)
o

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])